In [16]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [17]:
! wget https://raw.githubusercontent.com/motazsaad/login-ds/master/datasets/IRIS.csv

--2020-07-13 10:49:59--  https://raw.githubusercontent.com/motazsaad/login-ds/master/datasets/IRIS.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4466 (4.4K) [text/plain]
Saving to: ‘IRIS.csv.1’

IRIS.csv.1          100%[===================>]   4.36K  --.-KB/s    in 0s      

2020-07-13 10:49:59 (62.3 MB/s) - ‘IRIS.csv.1’ saved [4466/4466]



In [18]:
df = pd.read_csv('IRIS.csv')
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [19]:
df['species'] = pd.Categorical(df['species'])
df['species'] = df.species.cat.codes
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [20]:
val_df = df.sample(frac=0.2, random_state=1337)
train_df = df.drop(val_df.index)

print(
    "Using %d samples for training and %d for validation"
    % (len(train_df), len(val_df))
)

Using 120 samples for training and 30 for validation


In [21]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target, shuffle=True, batch_size=10):
  dataframe = dataframe.copy()
  labels = dataframe.pop(target)
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [22]:
train_ds = df_to_dataset(dataframe=train_df, target='species')
val_ds = df_to_dataset(dataframe=val_df,  target='species')

In [23]:
from tensorflow import feature_column

feature_columns = []
# numeric cols
for col in df.columns:
  if col == 'species':
    continue
  feature_columns.append(feature_column.numeric_column(col, dtype=tf.float16)) 
feature_columns

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float16, normalizer_fn=None)]

In [24]:
model = tf.keras.models.Sequential()

In [25]:
df.species.unique()

array([0, 1, 2], dtype=int8)

In [26]:
model.add(tf.keras.layers.DenseFeatures(feature_columns))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [27]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

12/12 [==============================] - 0s 16ms/step - loss: 1.0024 - accuracy: 0.3917 - val_loss: 0.9530 - val_accuracy: 0.3333
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 0.8488 - accuracy: 0.8333 - val_loss: 0.7382 - val_accuracy: 0.9000
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 0.7596 - accuracy: 0.6417 - val_loss: 0.6190 - val_accuracy: 0.9000
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 0.6627 - ac

In [31]:
model.evaluate(val_ds)

3/3 [==============================] - 0s 2ms/step - loss: 0.0591 - accuracy: 1.0000


[0.05905679240822792, 1.0]

In [32]:
model.predict(val_ds)

array([[9.98658657e-01, 1.34129089e-03, 3.94355881e-09],
       [3.06657384e-05, 4.65277061e-02, 9.53441620e-01],
       [3.22767012e-02, 9.63478446e-01, 4.24478855e-03],
       [1.76658004e-03, 9.63027418e-01, 3.52060162e-02],
       [3.69456611e-05, 1.24368131e-01, 8.75594914e-01],
       [7.40031919e-06, 2.48699486e-02, 9.75122690e-01],
       [3.19314694e-07, 4.54065250e-03, 9.95459080e-01],
       [1.14992568e-02, 9.86340523e-01, 2.16015172e-03],
       [9.99715149e-01, 2.84805341e-04, 2.19858728e-10],
       [1.42008475e-05, 3.26893926e-01, 6.73091888e-01],
       [9.96614516e-01, 3.38551728e-03, 1.32007543e-08],
       [9.98178482e-01, 1.82155950e-03, 4.75721951e-09],
       [9.93197799e-01, 6.80219149e-03, 3.88041670e-08],
       [9.98069704e-01, 1.93037314e-03, 1.03976978e-08],
       [9.99843001e-01, 1.56942697e-04, 4.85034408e-11],
       [9.99179661e-01, 8.20316607e-04, 1.81218740e-09],
       [5.30917669e-06, 4.58075739e-02, 9.54187095e-01],
       [9.95593131e-01, 4.40683